#2.155/6 Challenge Problem 2
<font size="1">
  Created by L. Regenwetter in Oct. 2023;   Artwork by Jessica Shung. </font>

In [38]:
%cd CP2_2023_public/

[Errno 2] No such file or directory: 'CP2_2023_public/'
/Users/zihaofoo/Documents/GitHub/CP2_2023_public


In [1]:
from utils_public import *
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis

ModuleNotFoundError: No module named 'PIL'

## Understanding City Zoning:
Cities are often comprised of districts. Typically, the city government zones areas allowing them some amount of control over what goes on in each distict. To simplify, we will be represing city districts as discrete spaces in a 7x7 grid. Each grid space will be filled with exactly one of five types of zones. We will discuss the districts below:

**Residential Zones [0]:** Residential zones are comprised primarily of housing and are where the workforce lives

**Industrial Zones [1]:** Industrial zones are the manufacturing centers of the city, packed with factories producing food and goods.

**Commercial Zones [2]:** Commercial zones are the retail hubs of the city, consisting of markets, stores, and restaurants.

**Park Zones [3]:** City parks are the green spaces of the city -- small tastes of nature within an urban environment.

**Office Zones [4]:** Office zones are the enterprise centers of global corporations and local businesses alike.

In [ ]:
plot_districts()

## The Data
The mayor has provided you access to a large dataset of 500,000 possible zoning configurations. Additionally, the mayor asked the advisors to score some of these zoning configurations according to their subjective opinions of efficacy. Since the advisors are "only human" they have only been able to label ~5,000 configurations each. Some of their assessments may seem questionable to you, but they mayor insists that the advisors are "experts" and their ratings should not be questioned.

Let's take a look at the data. First, lets have a look at the zoning layouts you are given.

In [40]:
grids = load_grids() #Helper function we have provided to load the grids from the dataset
grids.shape #Check shape

(500000, 7, 7)

Let's examine the first grid in the dataset. It consists of 7x7 entries, each ranging from 0 to 4. The entries denote the district occupying the grid space (labeled above).

In [ ]:
grids[0]

Let's visualize some of these grids. We have provided some utilities, mainly fucused on visualization, which we imported at the top of the notebook.

In [ ]:
plot_n_grids(grids[:10]) #Plotting function that plots some number of grids in a clean layout

Now lets have a look at the advisor scores:

In [41]:
ratings = np.load("datasets/scores.npy") #Load advisor scores
score_order = ["Wellness", "Tax", "Transportation", "Business"] #This is the order of the scores in the dataset
ratings_df = pd.DataFrame(ratings, columns = score_order) #Create a dataframe
display(ratings_df) #Print dataframe

,Wellness,Tax,Transportation,Business
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
499995,NaN,NaN,NaN,NaN
499996,NaN,NaN,NaN,NaN
499997,NaN,NaN,NaN,NaN
499998,NaN,NaN,NaN,NaN


We can see that the vast majority of ratings are NaN. After all, only 1% of the data is labeled. Let's confirm that there are the right number of NaNs in each column:

In [ ]:
ratings_df.isna().sum()

Let's plot the distribution over the scores that are rated. We see that each advisor's scores are fairly uniform from 0 to 1.

In [ ]:
plot_ratings_histogram(ratings)

## The Task:
**Satisfying the Advisors:**
You are tasked with finding zoning layouts that "satisfy" all four advisors, meaning that they each assign a score of at least 0.85. If even one of the advisors rates a layout under a 0.85, it will be rejected as a candidate. You much identify designs from the dataset or generate new designs that *you believe* will be accepted by the advisors.

**Identifying a diverse set:** The mayor has asked for a variety of "diverse" design candidates -- 100 city layouts to be exact -- which they will show to the advisors. The overall diversity of all valid (non-rejected) designs will be calculated. With this diversity metric, more designs is always better, so it is in your best interest to ensure that as few of your submitted designs are rejected as possible.

We have provided the function we will use to evaluate diversity. Here are a few tests to build some intuition with the diversity score.

### Building Intuition for Diversity

In [ ]:
diversity_score(grids[:100]) #Diversity of the first 100 grids in the dataset

In [ ]:
grids[:100].shape

In [ ]:
#If we set the top left corner to 0 in all grids, the diversity score should go down
g_1 = grids.copy()
g_1[:,0,0] = 0
diversity_score(g_1[:100])

In [ ]:
g_1[:,0,0].shape

In [ ]:
g_1[:100].shape

In [ ]:
#If some grids are the same, the diversity score should go down
g_2 = grids.copy()
g_2[0] = g_2[1]
diversity_score(g_2[:100])

In [ ]:
grids_flat = onehot_and_flatten(grids)
grids_flat.shape

In [ ]:
u = np.unique(grids_flat, axis=0)

In [ ]:
#If we submit fewer grids, the diversity score should go down (as will occur if invalid designs are submitted)
diversity_score(g_2[:99]) #Diversity of the first 99 grids in the dataset

In [ ]:
happy = np.random.rand(5,7,7)
print(happy)

In [ ]:
diverse_set = happy.argsort(0)
diverse_set

In [ ]:
diverse_set.shape

In [ ]:
# A diverse set of five grids where no two cities have the same type of district in the same grid space
diverse_set = np.random.rand(5,7,7).argsort(0)
plot_n_grids(diverse_set)
print(f"Diversity Score: {diversity_score(diverse_set, 5)}")


In [ ]:
#set of 5 grids where each district is independent and random with 20% probability for each district
random_set = np.random.randint(0,5, size=(5,7,7))
plot_n_grids(random_set)
print(f"Diversity Score: {diversity_score(random_set, 5)}")

In [ ]:
#Set of five grids where each grid space only takes one of two types across all cities
r1 = np.random.randint(0,5,size=(7,7))
r2 = np.random.randint(0,5,size=(7,7))
mask = np.random.randint(0,2,size=(5,7,7))
non_diverse_set = r1 * mask + r2 * (1-mask)
plot_n_grids(non_diverse_set)
print(f"Diversity Score: {diversity_score(non_diverse_set, 5)}")

In [ ]:
non_diverse_set.shape

Assuming that all designs in your submitted sets are valid, you will be able to *exactly calculate* the diversity of your submitted sets.

In [ ]:
grids = load_grids() #Helper function we have provided to load the grids from the dataset
grids.shape #Check shape

In [ ]:
ratings = np.load("datasets/scores.npy") #Load advisor scores
score_order = ["Wellness", "Tax", "Transportation", "Business"] #This is the order of the scores in the dataset
ratings_df = pd.DataFrame(ratings, columns = score_order) #Create a dataframe
display(ratings_df) #Print dataframe

In [42]:
grids_subset, ratings_subset = select_rated_subset(grids, ratings[:,0]) #gets subset of the dataset rated by advisor 0

In [ ]:
# grids_subset, ratings_subset = select_rated_subset(grids, ratings[:,2]) #gets subset of the dataset rated by advisor 0
# sorted_indices = np.argsort(ratings_subset)[2490:2510][::-1] 
# top_ten_grids = grids_subset[sorted_indices]
# plot_n_grids(top_ten_grids)
# ratings_subset[sorted_indices]

In [44]:
def compute_distance_to_class(grid, target_class):
    # Find the positions of the target class
    positions_target_class = np.argwhere(grid == target_class)
    
    # Initialize a distance matrix with large values
    distance_matrix = np.full(grid.shape, np.inf)
    
    for i in range(7):
        for j in range(7):
            for pos in positions_target_class:
                distance = np.linalg.norm(np.array([i, j]) - pos)
                distance_matrix[i, j] = min(distance_matrix[i, j], distance)
    
    return distance_matrix.flatten()

In [46]:
def compute_max_min_distance(grid, class_type_1, class_type_2):
    # Find the positions of the two class types
    positions_type_1 = np.argwhere(grid == class_type_1)
    positions_type_2 = np.argwhere(grid == class_type_2)

    # If either class type is not found in the grid, return None for min and max distances
    if len(positions_type_1) == 0 or len(positions_type_2) == 0:
        return None, None

    # Calculate all pairwise distances between the two sets of positions
    distances = []
    for pos1 in positions_type_1:
        for pos2 in positions_type_2:
            distance = np.linalg.norm(pos1 - pos2)
            distances.append(distance)

    # Return the minimum and maximum of the calculated distances
    return [float(min(distances)), float(max(distances))]

In [ ]:
grid_sample = np.random.randint(0, 5, (7, 7))
grid_sample

In [47]:
compute_max_min_distance(grid_sample, 1, 2)

[1.0, 7.810249675906654]

In [48]:
def count_connected_for_class(grid, target_class):
    height, width = grid.shape
    count = 0

    for i in range(height):
        for j in range(width):
            if grid[i][j] == target_class:
                # Check bottom neighbor
                if i + 1 < height and grid[i+1][j] == target_class:
                    count += 1

                # Check right neighbor
                if j + 1 < width and grid[i][j+1] == target_class:
                    count += 1

    return count

In [49]:
def compute_features(grid, advisor):

    features = []
    grid = grid.astype(int)
    # Number of each type
    counts = np.bincount(grid.flatten(), minlength=5)
    features.extend(counts)
    
    inter_adjacency = np.zeros((5, 5), dtype=int)
    
    for i in range(7):
        for j in range(7):
            current_val = grid[i, j]
            for dx, dy in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
                if 0 <= i+dx < 7 and 0 <= j+dy < 7:
                    neighbor_val = grid[i+dx, j+dy]
                    # Only update the upper diagonal (including main diagonal)
                    if current_val <= neighbor_val:
                        inter_adjacency[current_val, neighbor_val] += 1
                    
    features.extend(inter_adjacency[np.triu_indices(5, k=0)])

        # Create a dictionary to store distances for each unique class pair
    distances = {(i, j): [] for i in range(5) for j in range(i, 5)}

    # Compute distances for each pair of cells
    for i in range(7):
        for j in range(7):
            for m in range(7):
                for n in range(7):
                    d = np.sqrt((i - m) ** 2 + (j - n) ** 2)
                    class_pair = tuple(sorted([grid[i, j], grid[m, n]]))
                    distances[class_pair].append(d)

    # Compute statistics for each class pair and store in a flattened list
    flattened_stats = []
    # for key, values in distances.items():
    #     mean_val = np.mean(values)
    #     var_val = np.var(values)
    #     # skew_val = skew(values)
    #     # kurt_val = kurtosis(values)
    #     # flattened_stats.extend([mean_val, var_val, skew_val, kurt_val])        
    #     flattened_stats.extend([mean_val, var_val])
    # features.extend(flattened_stats)

    for key, values in distances.items():
        if values:  # Check if the list is not empty
            try:
                mean_val = np.mean(values)
                var_val = np.var(values)
            except:
                mean_val = 0  # or any default value
                var_val = 0  # or any default value
            flattened_stats.extend([mean_val, var_val])
        else:  # Handle the case where the list is empty
            flattened_stats.extend([0, 0])  # or any default value
    
    if advisor == 0:
        " Wellness advisor is focused on the health and wellbeing (both physical and mental) of citizens"
        " They are very invested in the quality and accessibility of city's green spaces."
        # Distance of park zones from each grid element
        distance_matrix = compute_distance_to_class(grid, target_class = 3)
        features.extend(distance_matrix)
        # Number of connected parks
        connections = count_connected_for_class(grid, target_class= 3)
        features.extend([connections])
        
    if advisor == 2:
        " Transportation advisor places an emphasis on accessibility and emissions. "
        " They are focused on mimizing the distance over which the workforce needs to commute."
        for cls in [0, 1, 2, 4]:
                   
            # obtains the minimum and max distance between residential areas and three others
            # max_min = compute_max_min_distance(grid, class_type_1 = 0, class_type_2 = cls)
            # features.extend(max_min)
            # obtains distance to nearest residential
            distance_matrix = compute_distance_to_class(grid, target_class = cls)
            features.extend(distance_matrix)

            # could possibly add proximity of houses at the centre
            # find whether the orientation matters
            

    return features

In [55]:
from multiprocessing import Pool, cpu_count
from functools import partial

def parallel_compute(grids, advisor):
    with Pool(cpu_count()) as p:
        # Partially apply the advisor to the compute_features function
        func = partial(compute_features, advisor=advisor)
        # Map the function over grids
        return np.array(p.starmap(func, [(grid,) for grid in grids]))

advisor_val = 2
features_subset = parallel_compute(grids_subset, advisor_val)

Process SpawnPoolWorker-42:
Traceback (most recent call last):
  File "/Users/zihaofoo/opt/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/zihaofoo/opt/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/zihaofoo/opt/anaconda3/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/zihaofoo/opt/anaconda3/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'compute_features' on <module '__main__' (built-in)>
Process SpawnPoolWorker-41:
Traceback (most recent call last):
  File "/Users/zihaofoo/opt/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/zihaofoo/opt/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File 

KeyboardInterrupt: 

In [54]:
# features_subset = compute_features(grids_subset, 0)
features_subset[np.isnan(features_subset)] = 0


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [51]:
from sklearn.model_selection import train_test_split
features_train, features_test, ratings_train, ratings_test = train_test_split(features_subset, ratings_subset)

NameError: name 'features_subset' is not defined

In [ ]:
#Convert to dataframe and specify dtype of object to ensure categorical handling of data
features_train = pd.DataFrame(features_train, columns = range(features_subset.shape[1]), dtype = "object")
features_test = pd.DataFrame(features_test, columns = range(features_subset.shape[1]), dtype = "object")

#Convert ratings to dataframe and label the column so we can tell Autogluon what to predict
preds_train = pd.DataFrame(ratings_train, columns = ["ratings"])

#Concatenate train grids and ratings together
all_train = pd.concat([features_train, preds_train], axis=1)
display(all_train) #Have a look to check things over

In [ ]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label="ratings").fit(all_train, hyperparameters = {'NN_TORCH':{'weight_decay':1e-4}, 'GBM':{}, 'RF':{}, 'XT':{}, 'KNN':{}})

In [ ]:
from sklearn.metrics import r2_score

preds_train = predictor.predict(features_train)
preds_test = predictor.predict(features_test)

def plot_and_r2(preds_train, preds_test, ratings_train, ratings_test, i):
    plt.scatter(ratings_train, preds_train, label='Train Set Preds', s=3, c = "#F08E18") #Train set in orange
    plt.scatter(ratings_test, preds_test, label='Test Set Preds', s=5, c = "#DC267F") #Test set in magenta
    plt.plot([0,1], [0,1], label="target", linewidth=3, c="k") # Target line in Black
    plt.xlabel("Actual Rating")
    plt.ylabel("Predicted Rating")
    plt.title(f"Advisor {i} Predictions")
    plt.legend()
    plt.show()
    print(f"Train Set R2 score: {r2_score(ratings_train, preds_train)}") #Calculate R2 score
    print(f"Test Set R2 score: {r2_score(ratings_test, preds_test)}")
plot_and_r2(preds_train, preds_test, ratings_train, ratings_test, 0)